On va tenter de simuler une pseudo carte scolaire. En effet, notre modélisation du lien entre taux de réussite au baccalauréat et revenu souffre probablement du fait qu'on ait associé le revenu des élèves à celui des communes de leur lycée. 
Or, des élèves viennent d'autres communes non pourvues d'établissement scolaire et le revenu médian peut être très hétérogène entre deux communes. 
On va donc essayer de refaire une carte scolaire de la manière suivante :
- Chaque commune sera associée à un point géographique. Si elle est dépourvue de lycée, on l'associera au lycée le plus proche. 
- On ponderera alors un revenu médian pour chaque lycée en fonction du revenu médian de chaque commune qui lui est associée et de la population des communes prises en compte. On posera donc l'hypothèse que la proportion de bacheliers potentiels est identique dans chaque commune (ce qui est discutable étant donné qu'il y'a une ségrégation spatiale générationnelle sur le territoire français)
- On procédera ainsi à une nouvelle régression avec les données obtenues

On doit donc créer une base de données adaptée. Il faut qu'on ait dedans
- Chaque établissement scolaire, avec son nombre de candidats et le taux de réussite par filières. 
- Chaque commune avec le revenu médian associé
- Les coordonnées géographiques des lycées et des communes 
En plus des bases de données utilisées précédemment, nous avons utilisé une base de données de La Poste pour avoir la longitude et la latitude de chaque commune. 

In [5]:
!pip install pathlib2
!pip install python-Levenshtein
!pip install --upgrade xlrd
!pip install git+https://github.com/InseeFrLab/Py-Insee-Data.git
import requests
url = 'https://github.com/InseeFrLab/Py-Insee-Data/archive/refs/heads/master.zip'
r = requests.get(url)
with open("pynsee.zip" , 'wb') as zipfile:
    zipfile.write(r.content)
!pip install --ignore-installed pynsee.zip
!pip install python-Levenshtein
!pip install openpyxl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pynsee.download
import seaborn as sns

  Cloning https://github.com/InseeFrLab/Py-Insee-Data.git to /tmp/pip-req-build-4x8iurw8
  Running command git clone --filter=blob:none --quiet https://github.com/InseeFrLab/Py-Insee-Data.git /tmp/pip-req-build-4x8iurw8
  Resolved https://github.com/InseeFrLab/Py-Insee-Data.git to commit 916d8bc82ea43f637047ed989fb7dbc491f56895
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Processing ./pynsee.zip
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached urllib3-1.26.13-py2.py3-none-any.whl (140 kB)
  Using cached Unidecode-1.3.6-py3-none-any.whl (235 kB)
  Using cached pandas-1.5.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.1 MB)
  Using cached Shapely-1.8.5.post1-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (2.0 MB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl (9.6 kB)

In [17]:
#Création de la base de données

#Base de communes
df_communes = pd.read_excel("FILO2019_DEC_COM.xlsx", sheet_name = 1, header=[4,5])
df_com = df_communes.copy()
df_com = df_com.drop(['PMIMP19', 'Q119', 'Q319','Q3_Q1','D119', 'D219', 'D319', 'D419', 'D619', 'D719', 'D819', 'D919', 'RD', 'S80S2019', 'GI19', 'PACT19', 'PTSA19', 'PCHO19', 'PBEN19', 'PPEN19', 'PAUT19'], axis = 1, level = 1 )

#Base des lycées
dflyc = pd.read_excel('base_def_SansDOMTOM.xlsx')

#Base des coordonnées géographiques des communes 
dfgeo = pd.read_csv('laposte_hexasmal.csv', sep=';')
dfgeo

,code_commune_insee,nom_de_la_commune,code_postal,ligne_5,libelle_d_acheminement,coordonnees_gps
0,01002,L ABERGEMENT DE VAREY,1640,NaN,L ABERGEMENT DE VAREY,"46.009605679,5.428087796"
1,01005,AMBERIEUX EN DOMBES,1330,NaN,AMBERIEUX EN DOMBES,"45.99616357,4.911967138"
2,01006,AMBLEON,1300,NaN,AMBLEON,"45.749886304,5.594584599"
3,01008,AMBUTRIX,1500,NaN,AMBUTRIX,"45.936683389,5.332446671"
4,01009,ANDERT ET CONDON,1300,NaN,ANDERT ET CONDON,"45.787194157,5.657799976"
...,...,...,...,...,...,...
39188,98820,OUVEA,98814,OUVEA,FAYAOUE,NaN
39189,98826,POUM,98826,NaN,POUM,NaN
39190,98831,VOH,98883,VOH,OUACO,NaN
39191,98833,KOUAOUA,98818,NaN,KOUAOUA,NaN
